# 1. Imports

In [ ]:
# !pip install adversarial-robustness-toolbox

: 

In [9]:
import numpy as np
import math
from random import random, randint

import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt

from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Softmax

from gurobipy import *

import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 2. Compute statistical metrics

In [10]:
ACAS_labels = ["COC","WR","WL","SR","SL"]

In [ ]:
# 1 - Load the model ACAS

def load_ACAS(num_net=(1, 1), folder="ACAS_XU_tf_keras/"):
    if (num_net[0] in range(1,6)) and (num_net[1] in range(1,10)):
        path_model = folder + "ACASXU_{}_{}.h5".format(num_net[0], num_net[1])
        model = load_model(path_model)
        model.compile()
    else:
        raise Exception("ACAS neural network {0}-{1} doesn't exist.".format(num_net[0],num_net[1]))
    
    return model

In [ ]:
# 2 - Compute manually the output of each layer on the initial point

def compute_layers(model,x_):
    y_=[[]]
    for j in range(model.layers[0].weights[0].shape[1]):
        z=0
        for k in range(model.layers[0].weights[0].shape[0]):
            z+=model.layers[0].weights[0][k][j]* x_[k]
        z+=model.layers[0].weights[1][j]
        y_[0].append(z)

    for i in range(1,len(model.layers)-1): # We skip last layer (softmax)
        y_.append([])
        if model.layers[i].__class__.__name__=='Activation':
            for j in range(len(y_[i-1])):
                if y_[i-1][j]<0:
                    y_[i].append(0)
                else:
                    y_[i].append(y_[i-1][j])
        elif model.layers[i].__class__.__name__=='Dense':
            for j in range(model.layers[i].weights[0].shape[1]):
                z=0
                for k in range(model.layers[i].weights[0].shape[0]):
                    z+=model.layers[i].weights[0][k][j]* y_[i-1][k]
                z+=model.layers[i].weights[1][j]
                y_[i].append(z)
    
    # 3 - Verify that the result is the same as the answer of the network
    
    #pdb.set_trace()
    #model.summary()
    # print(model.predict(np.expand_dims(x_, axis=0)))
    # lastLayer = Softmax()
    # print(lastLayer(y_[-1]).numpy())
    
    return y_

In [39]:
# We use linear programming to compute the approximation of epsilon
def compute_epsilon(x_, y_, l, model):
    """Do not forget to set folder and, if you use Colab, to import your neural networks!"""
    
    # 4 - Create model and variables
    
    m = Model("rho")
    t=m.addVar(lb=0.0, ub=float('inf'), obj=0.0, vtype=GRB.CONTINUOUS, name = f't')
    x=[m.addVar(lb=-float('inf'), ub=float('inf'), vtype=GRB.CONTINUOUS, name = f'x_{i}') for i in range(len(x_))]
    y=[[m.addVar(lb=-float('inf'), ub=float('inf'), vtype=GRB.CONTINUOUS, name = f'y_{i}_{j}') for j in range(len(y_[i]))] for i in range(len(model.layers)-1)]


    # 5 - Add constraints

    # 5.1 - Constraint |x-x_|oo<=t
    for i in range(len(x_)):
        m.addConstr(x[i]-x_[i] <= t)
        m.addConstr(-x[i]+x_[i] <= t)

    # 5.2 - Neural Network Constraint
    for j in range(model.layers[0].weights[0].shape[1]):
        z=LinExpr()
        for k in range(model.layers[0].weights[0].shape[0]):
            z+=model.layers[0].weights[0][k][j]* x[k]
        z+=model.layers[0].weights[1][j]
        m.addConstr(y[0][j] == z)

    for i in range(1,len(model.layers)-1): #Skip last layer (softmax)
        if model.layers[i].__class__.__name__=='Activation':
            for j in range(len(y_[i-1])):
                if y_[i-1][j]<0:
                    m.addConstr(y[i][j] == 0)
                else:
                    m.addConstr(y[i][j] == y[i-1][j])
        elif model.layers[i].__class__.__name__=='Dense':
            for j in range(model.layers[i].weights[0].shape[1]):
                z=LinExpr()
                for k in range(model.layers[i].weights[0].shape[0]):
                    z+=model.layers[i].weights[0][k][j]* y[i-1][k]
                z+=model.layers[i].weights[1][j]
                m.addConstr(y[i][j] == z)

    # # 5.3 - Target output label
    for i in range(len(y[-1])):
        if i != l:
            m.addConstr(y[-1][i]<=y[-1][l])

    # 6 - Objective : minimise distance to inital point
    
    m.setObjective(t, GRB.MINIMIZE)

    # 7 - Optimize

    m.params.outputflag = 0 # mute mode
    m.update()
    m.display()
    m.optimize()

    # 8 - Read result
    if  m.status == GRB.INFEASIBLE:
        print("No solution !!")

        return 
    else:
        lastLayer = Softmax()
        print("Point : ",x_," -> ",[z.x for z in x])
        print("Scores : ",lastLayer(y_[-1]).numpy()," -> ",lastLayer([z.x for z in y[-1]]).numpy())
        print("Objective : ",m.objVal)

        return m.objVal

1/1 [==============================] - 0s 476ms/step
[[0.19943528 0.20016503 0.20013963 0.20013508 0.20012508]]
[0.19943528 0.20016503 0.20013963 0.20013508 0.20012508]
Initial point :  [1 1 0 0 0]
Initial scores :  [0.19943528 0.20016503 0.20013963 0.20013508 0.20012508]
Final point :  [0.5554996229279519, 0.5554996229279519, 0.44450037707204815, -0.44450037707204815, 0.44450037707204815]
Final scores :  [0.20004897 0.19997278 0.19996646 0.19996282 0.20004897]
Objective :  0.44450037707204815


0.44450037707204815

In [ ]:
def compute_rho(X,model):
  R=[]
  for x_ in X:
    R.append([])
    y_=compute_layers(model,x_)
    l=np.argmax(y_[-1])
    m=float('inf')
    for i in range(5):
      eps=compute_epsilon(x_, y_, i, model)
      R[-1].append(eps)
      if i!=l:
        m=min(m,eps)
    R[-1].append(m)
  return R

In [ ]:
def compute_metrics(R,eps):
  phi,mu=[0]*6,[0]*6
  for i in range(6):
    n=0
    for rho in R:
      if rho[i]<=eps:
        n+=1
        mu[i]+=rho
  
    phi[i]=n/len(R)
    mu[i]/=n

  return phi,mu